<a href="https://colab.research.google.com/github/AndrewStaines/AIML/blob/main/Pytorch_vs_Keras.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision.datasets as datasets
import torchvision.transforms as transforms
import torchvision.datasets as MNIST
import tensorflow as tf

from torch.utils.data import DataLoader
from tensorflow.keras import layers,models
from tensorflow.keras.datasets import mnist
from tensorflow.keras.utils import to_categorical

In [ ]:
# Load and preprocess the MNIST dataset using PyTorch
transform = transforms.Compose([transforms.ToTensor(), transforms.Normalize((0.5,), (0.5,))])
train_dataset = datasets.MNIST(root='./data', train=True, download=True, transform=transform)
test_dataset = datasets.MNIST(root='./data', train=False, download=True, transform=transform)

In [ ]:
train_loader = DataLoader(dataset=train_dataset, batch_size=64, shuffle=True)
test_loader = DataLoader(dataset=test_dataset, batch_size=64, shuffle=False)

In [ ]:
(train_images, train_labels), (test_images, test_labels) = mnist.load_data()

In [ ]:
train_images = train_images.reshape((60000, 28, 28, 1)).astype('float32') / 255
test_images = test_images.reshape((10000, 28, 28, 1)).astype('float32') / 255

In [ ]:
train_labels = to_categorical(train_labels)
test_labels = to_categorical(test_labels)

In [ ]:
class Net(nn.Module):
  def __init__(self):
    super(Net,self).__init__()
    self.fc1=nn.Linear(28*28,128)
    self.fc2=nn.Linear(128,64)
    self.fc3=nn.Linear(64,32)
    self.fc4=nn.Linear(32,16)
    self.fc5=nn.Linear(16,8)
    self.fc6=nn.Linear(8,10)
  def forward(self,x):
    x=x.view(-1,28*28)
    x=torch.relu(self.fc1(x))
    x=torch.relu(self.fc2(x))
    x=torch.relu(self.fc3(x))
    x=torch.relu(self.fc4(x))
    x=torch.relu(self.fc5(x))
    x=self.fc6(x)
    return x

In [ ]:
model_pt = Net()
criterion = nn.CrossEntropyLoss()
optimizer_pt = optim.Adam(model_pt.parameters(), lr=0.001)

In [ ]:
import time
start_time_tf = time.time()
model_tf = models.Sequential()
model_tf.add(layers.Flatten(input_shape=(28, 28, 1)))
model_tf.add(layers.Dense(128, activation='relu'))
model_tf.add(layers.Dense(10, activation='softmax'))

In [ ]:
model_tf.compile(optimizer='adam',
                 loss='categorical_crossentropy',
                 metrics=['accuracy'])

In [ ]:
model_tf.fit(train_images, train_labels, epochs=5, batch_size=64, validation_split=0.2)
end_time_tf = time.time()

Epoch 1/5
750/750 [==============================] - 6s 8ms/step - loss: 0.3265 - accuracy: 0.9096 - val_loss: 0.1790 - val_accuracy: 0.9485
Epoch 2/5
750/750 [==============================] - 4s 5ms/step - loss: 0.1534 - accuracy: 0.9556 - val_loss: 0.1343 - val_accuracy: 0.9626
Epoch 3/5
750/750 [==============================] - 4s 6ms/step - loss: 0.1078 - accuracy: 0.9688 - val_loss: 0.1183 - val_accuracy: 0.9644
Epoch 4/5
750/750 [==============================] - 6s 7ms/step - loss: 0.0825 - accuracy: 0.9758 - val_loss: 0.1118 - val_accuracy: 0.9673
Epoch 5/5
750/750 [==============================] - 4s 5ms/step - loss: 0.0643 - accuracy: 0.9814 - val_loss: 0.0962 - val_accuracy: 0.9713


In [ ]:
print(f"TensorFlow Training Time: {end_time_tf - start_time_tf} seconds")

TensorFlow Training Time: 42.0209105014801 seconds


In [ ]:
start_time_pt = time.time()
for epoch in range(5):
    for images, labels in train_loader:
        optimizer_pt.zero_grad()
        outputs = model_pt(images.view(-1, 28 * 28))
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer_pt.step()
end_time_tf = time.time()
print(f"PyTorch Training Time: {end_time_tf - start_time_tf} seconds")

PyTorch Training Time: 144.44703602790833 seconds


In [ ]:
# Evaluation using TensorFlow
test_loss_tf, test_acc_tf = model_tf.evaluate(test_images, test_labels)
print(f"TensorFlow Test Accuracy: {test_acc_tf}")

313/313 [==============================] - 1s 2ms/step - loss: 0.0899 - accuracy: 0.9711
TensorFlow Test Accuracy: 0.9710999727249146


In [ ]:
# Evaluation using PyTorch
correct = 0
total = 0
with torch.no_grad():
    for images, labels in test_loader:
        outputs = model_pt(images.view(-1, 28 * 28))
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

accuracy_pt = correct / total
print(f"PyTorch Test Accuracy: {accuracy_pt}")

PyTorch Test Accuracy: 0.9593
